In [323]:
import pandas as pd

In [324]:
date_start = input('Date de départ des données ? ')
date_saisson_id = '2' + date_start
date_saisson_id = int(date_saisson_id)
date_start = int(date_start)
print(type(date_saisson_id))

<class 'int'>


In [325]:

df_game_base = pd.read_csv('game.csv')
df_game_base = df_game_base.dropna(how='all', axis=0, subset=['wl_home'])
# on garde que la "Regular Season"
df_game = df_game_base[df_game_base['season_type'].isin(['Regular Season'])]
# on garde que les années supérieures à 2010
df_game= df_game[df_game['season_id'] >= date_saisson_id]
# on garde que les colonnes suivantes : 'season_id', 'team_name_home', 'wl_home', 'team_name_away', 'wl_away'
df_game = df_game.loc[:,['season_id', 'team_name_home', 'wl_home', 'team_name_away', 'wl_away']]
df_game

,season_id,team_name_home,wl_home,team_name_away,wl_away
36041,22000,Dallas Mavericks,W,Milwaukee Bucks,L
36042,22000,Atlanta Hawks,L,Charlotte Hornets,W
36043,22000,Chicago Bulls,L,Sacramento Kings,W
36044,22000,Toronto Raptors,L,Detroit Pistons,W
36045,22000,Orlando Magic,W,Washington Wizards,L
...,...,...,...,...,...
65537,22022,New York Knicks,L,Indiana Pacers,W
65538,22022,Brooklyn Nets,L,Philadelphia 76ers,W
65539,22022,Chicago Bulls,W,Detroit Pistons,L
65540,22022,Toronto Raptors,W,Milwaukee Bucks,L


In [326]:
# conversion de season_id en année
def supprimer_premier_2(valeur):
    valeur_str = str(valeur)
    return int(valeur_str.replace('2', '', 1)) if '2' in valeur_str else valeur

df_game['season'] = df_game['season_id'].apply(supprimer_premier_2)
df_game['season'] = pd.to_datetime(df_game['season'], format='%Y').dt.year
df_game

,season_id,team_name_home,wl_home,team_name_away,wl_away,season
36041,22000,Dallas Mavericks,W,Milwaukee Bucks,L,2000
36042,22000,Atlanta Hawks,L,Charlotte Hornets,W,2000
36043,22000,Chicago Bulls,L,Sacramento Kings,W,2000
36044,22000,Toronto Raptors,L,Detroit Pistons,W,2000
36045,22000,Orlando Magic,W,Washington Wizards,L,2000
...,...,...,...,...,...,...
65537,22022,New York Knicks,L,Indiana Pacers,W,2022
65538,22022,Brooklyn Nets,L,Philadelphia 76ers,W,2022
65539,22022,Chicago Bulls,W,Detroit Pistons,L,2022
65540,22022,Toronto Raptors,W,Milwaukee Bucks,L,2022


In [327]:
# total des matchs 'away' gagnés par saison
df_new_game_away = df_game.groupby(['team_name_away', 'season'])['wl_away'].value_counts().reset_index(name='total_away')
df_new_game_away = df_new_game_away.sort_values(by=['team_name_away', 'season', 'wl_away'])
# on conserve que les matchs gagnés
df_new_game_away = df_new_game_away.loc[df_new_game_away['wl_away'] == 'W'].reset_index()
df_new_game_away

,index,team_name_away,season,wl_away,total_away
0,1,Atlanta Hawks,2000,W,7
1,3,Atlanta Hawks,2001,W,10
2,5,Atlanta Hawks,2002,W,9
3,7,Atlanta Hawks,2003,W,10
4,9,Atlanta Hawks,2004,W,4
...,...,...,...,...,...
651,1303,Washington Wizards,2018,W,10
652,1305,Washington Wizards,2019,W,9
653,1307,Washington Wizards,2020,W,15
654,1309,Washington Wizards,2021,W,14


In [328]:
# total des matchs 'home' gagnés par saison
df_new_game_home = df_game.groupby(['team_name_home', 'season'])['wl_home'].value_counts().reset_index(name='total_home')
df_new_game_home = df_new_game_home.sort_values(by=['team_name_home', 'season', 'wl_home'])
# on conserve que les matchs gagnés
df_new_game_home = df_new_game_home.loc[df_new_game_home['wl_home'] == 'W'].reset_index()
df_new_game_home

,index,team_name_home,season,wl_home,total_home
0,1,Atlanta Hawks,2000,W,18
1,2,Atlanta Hawks,2001,W,23
2,4,Atlanta Hawks,2002,W,26
3,7,Atlanta Hawks,2003,W,18
4,9,Atlanta Hawks,2004,W,9
...,...,...,...,...,...
651,1302,Washington Wizards,2018,W,22
652,1305,Washington Wizards,2019,W,16
653,1306,Washington Wizards,2020,W,19
654,1308,Washington Wizards,2021,W,21


In [329]:
df_new_game_home.insert(loc=len(df_new_game_home.columns), column='total_away', value=df_new_game_away['total_away'])
df_new_game_home

,index,team_name_home,season,wl_home,total_home,total_away
0,1,Atlanta Hawks,2000,W,18,7
1,2,Atlanta Hawks,2001,W,23,10
2,4,Atlanta Hawks,2002,W,26,9
3,7,Atlanta Hawks,2003,W,18,10
4,9,Atlanta Hawks,2004,W,9,4
...,...,...,...,...,...,...
651,1302,Washington Wizards,2018,W,22,10
652,1305,Washington Wizards,2019,W,16,9
653,1306,Washington Wizards,2020,W,19,15
654,1308,Washington Wizards,2021,W,21,14


In [330]:
colomns_list = ['total_home', 'total_away']
df_new_game_home['win_match'] = df_new_game_home[colomns_list].sum(axis=1)
df_new_game_home

,index,team_name_home,season,wl_home,total_home,total_away,win_match
0,1,Atlanta Hawks,2000,W,18,7,25
1,2,Atlanta Hawks,2001,W,23,10,33
2,4,Atlanta Hawks,2002,W,26,9,35
3,7,Atlanta Hawks,2003,W,18,10,28
4,9,Atlanta Hawks,2004,W,9,4,13
...,...,...,...,...,...,...,...
651,1302,Washington Wizards,2018,W,22,10,32
652,1305,Washington Wizards,2019,W,16,9,25
653,1306,Washington Wizards,2020,W,19,15,34
654,1308,Washington Wizards,2021,W,21,14,35


In [331]:
# dataframe du total des matchs gagnés par équipe par année

df_new_game = df_new_game_home.loc[:,['team_name_home', 'season', 'win_match']]
df_new_game['Team'] = df_new_game['team_name_home']
df_new_game = df_new_game.loc[:,['Team', 'season', 'win_match']]
df_new_game

,Team,season,win_match
0,Atlanta Hawks,2000,25
1,Atlanta Hawks,2001,33
2,Atlanta Hawks,2002,35
3,Atlanta Hawks,2003,28
4,Atlanta Hawks,2004,13
...,...,...,...
651,Washington Wizards,2018,32
652,Washington Wizards,2019,25
653,Washington Wizards,2020,34
654,Washington Wizards,2021,35


In [332]:
df_draft_history = pd.read_csv('draft_history.csv')

In [333]:

delta = input('Décalage de date ? ') 
delta = int(delta)
df_draft_history['Team']  = df_draft_history['team_city'] + ' ' + df_draft_history['team_name']
#ajoute d'un décalage de 4 ans
df_draft_history['season'] = df_draft_history.apply(lambda x: x['season'] + delta, axis=1)
# on garde que les années supérieures à 2010
df_draft_history= df_draft_history[df_draft_history['season'] >= date_start]
df_draft_history

,person_id,player_name,season,round_number,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag,Team
6394,947,Allen Iverson,2000,1,1,1,Draft,1610612755,Philadelphia,76ers,PHI,Georgetown,College/University,1,Philadelphia 76ers
6395,948,Marcus Camby,2000,1,2,2,Draft,1610612761,Toronto,Raptors,TOR,Massachusetts,College/University,1,Toronto Raptors
6396,949,Shareef Abdur-Rahim,2000,1,3,3,Draft,1610612763,Vancouver,Grizzlies,VAN,California,College/University,1,Vancouver Grizzlies
6397,950,Stephon Marbury,2000,1,4,4,Draft,1610612749,Milwaukee,Bucks,MIL,Georgia Tech,College/University,1,Milwaukee Bucks
6398,951,Ray Allen,2000,1,5,5,Draft,1610612750,Minnesota,Timberwolves,MIN,Connecticut,College/University,1,Minnesota Timberwolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,1641771,Jalen Slawson,2027,2,24,54,Draft,1610612758,Sacramento,Kings,SAC,Furman,College/University,1,Sacramento Kings
7986,1631209,Isaiah Wong,2027,2,25,55,Draft,1610612754,Indiana,Pacers,IND,Miami (FL),College/University,1,Indiana Pacers
7987,1641844,Tarik Biberovic,2027,2,26,56,Draft,1610612763,Memphis,Grizzlies,MEM,Fenerbahce S.K. (Turkey),Other Team/Club,1,Memphis Grizzlies
7988,1631218,Trayce Jackson-Davis,2027,2,27,57,Draft,1610612764,Washington,Wizards,WAS,Indiana,College/University,1,Washington Wizards


In [334]:
df_draft_history = df_draft_history.loc[:, ['player_name', 'season', 'Team', 'overall_pick']]
df_draft_history

,player_name,season,Team,overall_pick
6394,Allen Iverson,2000,Philadelphia 76ers,1
6395,Marcus Camby,2000,Toronto Raptors,2
6396,Shareef Abdur-Rahim,2000,Vancouver Grizzlies,3
6397,Stephon Marbury,2000,Milwaukee Bucks,4
6398,Ray Allen,2000,Minnesota Timberwolves,5
...,...,...,...,...
7985,Jalen Slawson,2027,Sacramento Kings,54
7986,Isaiah Wong,2027,Indiana Pacers,55
7987,Tarik Biberovic,2027,Memphis Grizzlies,56
7988,Trayce Jackson-Davis,2027,Washington Wizards,57


In [335]:
df_common_player_info = pd.read_csv('common_player_info.csv')

In [336]:

df_common_player_info = df_common_player_info.loc[:,['position', 'display_first_last']]
df_common_player_info = df_common_player_info.fillna('other')
df_common_player_info

,position,display_first_last
0,Forward,Alaa Abdelnaby
1,Center,Kareem Abdul-Jabbar
2,Forward-Guard,Tariq Abdul-Wahad
3,Forward,Shareef Abdur-Rahim
4,Forward,Tom Abernethy
...,...,...
4166,Forward,Paul Zipser
4167,Center,Ante Zizic
4168,Center,Jim Zoet
4169,Center,Ivica Zubac


In [337]:
df_draft = df_draft_history.merge(df_common_player_info, left_on='player_name', right_on='display_first_last')
df_draft

,player_name,season,Team,overall_pick,position,display_first_last
0,Allen Iverson,2000,Philadelphia 76ers,1,Guard,Allen Iverson
1,Marcus Camby,2000,Toronto Raptors,2,Center-Forward,Marcus Camby
2,Shareef Abdur-Rahim,2000,Vancouver Grizzlies,3,Forward,Shareef Abdur-Rahim
3,Stephon Marbury,2000,Milwaukee Bucks,4,Guard,Stephon Marbury
4,Antoine Walker,2000,Boston Celtics,6,Forward,Antoine Walker
...,...,...,...,...,...,...
1162,Vince Williams Jr.,2026,Memphis Grizzlies,47,Guard,Vince Williams Jr.
1163,Kendall Brown,2026,Minnesota Timberwolves,48,Guard,Kendall Brown
1164,Isaiah Mobley,2026,Cleveland Cavaliers,49,Forward,Isaiah Mobley
1165,Tyrese Martin,2026,Golden State Warriors,51,Guard,Tyrese Martin


In [338]:
# dataframe des joueurs avec leur position et numéro de draft
df_draft = df_draft.loc[:, ['overall_pick', 'season', 'position', 'Team']]
df_draft

,overall_pick,season,position,Team
0,1,2000,Guard,Philadelphia 76ers
1,2,2000,Center-Forward,Toronto Raptors
2,3,2000,Forward,Vancouver Grizzlies
3,4,2000,Guard,Milwaukee Bucks
4,6,2000,Forward,Boston Celtics
...,...,...,...,...
1162,47,2026,Guard,Memphis Grizzlies
1163,48,2026,Guard,Minnesota Timberwolves
1164,49,2026,Forward,Cleveland Cavaliers
1165,51,2026,Guard,Golden State Warriors


In [339]:
merged_df = pd.merge(df_new_game, df_draft, on=['Team', 'season'], how='outer')
merged_df

,Team,season,win_match,overall_pick,position
0,Atlanta Hawks,2000,25.0,28.0,Center
1,Atlanta Hawks,2001,33.0,22.0,Guard
2,Atlanta Hawks,2002,35.0,20.0,Forward
3,Atlanta Hawks,2003,28.0,10.0,Guard
4,Atlanta Hawks,2003,28.0,20.0,Forward-Guard
...,...,...,...,...,...
1314,Washington Wizards,2023,NaN,9.0,Forward
1315,Washington Wizards,2024,NaN,37.0,Guard
1316,Washington Wizards,2025,NaN,15.0,Forward
1317,Washington Wizards,2026,NaN,10.0,Guard


In [340]:
# merged_df['win_match'] = merged_df['win_match'].interpolate()
merged_df['overall_pick'] = merged_df['overall_pick'].fillna(61)
merged_df.loc[merged_df['overall_pick'] == 61, 'position'] = 'Nothing'
merged_df

,Team,season,win_match,overall_pick,position
0,Atlanta Hawks,2000,25.0,28.0,Center
1,Atlanta Hawks,2001,33.0,22.0,Guard
2,Atlanta Hawks,2002,35.0,20.0,Forward
3,Atlanta Hawks,2003,28.0,10.0,Guard
4,Atlanta Hawks,2003,28.0,20.0,Forward-Guard
...,...,...,...,...,...
1314,Washington Wizards,2023,NaN,9.0,Forward
1315,Washington Wizards,2024,NaN,37.0,Guard
1316,Washington Wizards,2025,NaN,15.0,Forward
1317,Washington Wizards,2026,NaN,10.0,Guard


In [341]:
merged_df.to_csv('dataset_NBA.csv')